In [1]:
# run: ipython kernel install --name "poetry-venv" --user

In [2]:
import random
from cqftsn5g.modules.Models import Flow, Path, Node, Link
def generate_flows(period_range, payload_range, priority_range, jitter_range, bandwidth_range, flow_type, path_range, numbers):
    # random select period, payload, priority, jitter, bandwidth to generate numbers flows
    flows = []
    
    for i in range(numbers):
        period = random.choice(period_range)
        payload = random.choice(payload_range)
        priority = random.choice(priority_range)
        jitter = random.choice([x for x in jitter_range if x < period])
        bandwidth = random.choice(bandwidth_range)
        latency = period 
        path = random.choice(path_range)
        id = f"{flow_type}_{i}"

        flows.append(Flow(id, period, payload, priority, latency, jitter, bandwidth, flow_type, path))

    return flows
    

In [3]:

# Nodes
host1 = Node("TSN host 1", True)
host2 = Node("TSN host 2", True)
tsn_sw1 = Node("TSN switch 1", False)
dstt = Node("DS-TT", False)
ue1 = Node("UE 1", False)
gnb = Node("gNodeB", False)
upf_nwtt = Node("UPF NW-TT", False)
tsn_sw2 = Node("TSN switch 2", False)
host3 = Node("TSN host 3", True)

# Links
host1_tsn_sw1 = Link(host1, tsn_sw1, 1000, "wire")
host2_tsn_sw1 = Link(host2, tsn_sw1, 1000, "wire")
tsn_sw1_dstt = Link(tsn_sw1, dstt, 1000, "wire")
dstt_ue1 = Link(dstt, ue1, 1000, "wire")
ue1_gnb = Link(ue1, gnb, 1000, "wireless")
gnb_upf_nwtt = Link(gnb, upf_nwtt, 1000, "wire")
upf_nwtt_tsn_sw2 = Link(upf_nwtt, tsn_sw2, 1000, "wire")
tsn_sw2_host3 = Link(tsn_sw2, host3, 1000, "wire")

links_TSN = [host1_tsn_sw1, host2_tsn_sw1, tsn_sw1_dstt, dstt_ue1, gnb_upf_nwtt, upf_nwtt_tsn_sw2, tsn_sw2_host3]
links_5G = [ue1_gnb]


In [4]:
Paths = {
    "path1": Path(
        src=host1,
        dst=host3,
        links=[
            host1_tsn_sw1,
            tsn_sw1_dstt,
            dstt_ue1,
            ue1_gnb,
            gnb_upf_nwtt,
            upf_nwtt_tsn_sw2,
            tsn_sw2_host3,
        ],
        cqi=14,
        direction="UL",
    ),
    "path2": Path(
        src=host2,
        dst=host3,
        links=[
            host2_tsn_sw1,
            tsn_sw1_dstt,
            dstt_ue1,
            ue1_gnb,
            gnb_upf_nwtt,
            upf_nwtt_tsn_sw2,
            tsn_sw2_host3,
        ],
        cqi=13,
        direction="UL",
    ),
    "path3": Path(
        src=host1,
        dst=host3,
        links=[
            tsn_sw2_host3,
            upf_nwtt_tsn_sw2,
            gnb_upf_nwtt,
            ue1_gnb,
            dstt_ue1,
            tsn_sw1_dstt,
            host1_tsn_sw1,
        ],
        cqi=12,
        direction="DL",
    ),
}

In [5]:
# period_range = [2000, 2500, 4000, 5000, 8000, 10000, 16000]
# payload_range = [50, 100, 250, 500]
# priority_range = [2, 3, 4]
# jitter_range = [500, 1000, 2000, 4000, 8000, 16000]

period_range = [4000, 5000, 10000, 20000]
payload_range = [50, 100, 250, 500]
priority_range = [2, 3, 4]
jitter_range = [2000, 2500, 5000]
path_range = ["path1", "path2", "path3"]

tt_flows = generate_flows(period_range, payload_range, priority_range, jitter_range, [0], "TT", path_range, 10) 


In [6]:
# period_range = [10000, 16000, 20000, 32000]
# payload_range = [800, 1000, 1200, 1500]
# priority_range = [5, 6, 7]
# jitter_range = [4000, 5000, 10000]
# bandwidth_range = [5000, 10000, 12000, 15000, 20000] # KB/sec
period_range = [10000, 20000]
payload_range = [500, 1000] # bytes
priority_range = [5, 6, 7]
jitter_range = [5000, 10000]
bandwidth_range = [20000, 40000]  # Mbps: 1Mbps = 1000000 bits per second = 125 000 bytes per second
path_range = ["path1", "path2", "path3"]

avb_flows = generate_flows(
    period_range, payload_range, priority_range, jitter_range, bandwidth_range, "AVB", path_range, 10
)


In [7]:
# from dataclasses import asdict
# from pprint import pprint
# import json

# pprint(tt_flows)
# flows = [asdict(flow) for flow in tt_flows + avb_flows]
# output = json.dump(flows, open("flows.json", "w"))

In [8]:
period_range = [5000]
payload_range = [100, 200, 300]
priority_range = [2, 3, 4]
jitter_range = [2500]

path_range = ["path1"]

test_flows = generate_flows(
    period_range, payload_range, priority_range, jitter_range, [0], "TT", path_range, 10
)


In [9]:
# GCD of the flow
import math

flows = test_flows

def calculate_gcd(flows: list[Flow]):
    periods = [flow.period for flow in flows]
    return math.gcd(*periods)

def calculate_lcm(flows: list[Flow]):
    periods = [flow.period for flow in flows]
    return math.lcm(*periods)

gcd_flows = calculate_gcd(flows)
lcm_flows = calculate_lcm(flows)


print(f"GCD: {gcd_flows}")
print(f"LCM: {lcm_flows}")
jitter_min = min([flow.jitter for flow in flows])
print(f"Jitter minimum: {jitter_min}")
latency_min = min([flow.latency for flow in flows])
print(f"Deadline minimum: {latency_min}")
hops_max = max([path.hops for key, path in Paths.items()])
print(f"hops maximum: {hops_max}")
interval_max = latency_min / hops_max
print(f"Interval maximum = deadline minimum / hops maximum: {latency_min / hops_max}")

interval_bound  = math.floor(min(jitter_min / 2, interval_max))

def find_largest_factor(gcd, interval_bound):
    for i in range(interval_bound - 1, 0, -1):
        if gcd % i == 0:
            return i
    return 1

# time_interval = find_largest_factor(gcd_flows, interval_bound)
hyper_cycle = lcm_flows * 2
time_interval = 500
print(f"Time interval: {time_interval}")

GCD: 5000
LCM: 5000
Jitter minimum: 2500
Deadline minimum: 10000
hops maximum: 7
Interval maximum = deadline minimum / hops maximum: 1428.5714285714287
Time interval: 500


In [10]:
from cqftsn5g.cp import cp_sat_scheduling
from cqftsn5g.modules.NRAmc import SlotFormat

mu = 4
U_TSN = 0.8
B_TSN = 1000 
U_5G = 0.8
num_bands = 50
slotFormat = SlotFormat(
    tdd=True,
    numDlSymbols=6,
    numUlSymbols=6,
    numFlexSymbols=2,
)

avb_flows = []

cp_sat_scheduling(test_flows, avb_flows, links_TSN, links_5G, Paths, hyper_cycle, time_interval, mu, U_TSN, B_TSN, U_5G, num_bands, slotFormat)

N = 20
TTI = 62.5
TSN capacity: 50000
5G # of RB: 320
UL payload sum: 2000, DL payload sum: 0
UL RB sum: 470, DL RB sum: 0
Solution found:
TT_0 is scheduled
ID: TT_0, period = 5000, payload = 100, priority = 4, deadline = 10000, jitter = 2500, bandwidth = 0, path = path1
1-th at interval 9
2-th at interval 15
TT_1 is scheduled
ID: TT_1, period = 5000, payload = 300, priority = 4, deadline = 10000, jitter = 2500, bandwidth = 0, path = path1
1-th at interval 4
2-th at interval 18
TT_2 is scheduled
ID: TT_2, period = 5000, payload = 100, priority = 3, deadline = 10000, jitter = 2500, bandwidth = 0, path = path1
1-th at interval 1
2-th at interval 18
TT_3 is scheduled
ID: TT_3, period = 5000, payload = 300, priority = 4, deadline = 10000, jitter = 2500, bandwidth = 0, path = path1
1-th at interval 8
2-th at interval 18
TT_4 is scheduled
ID: TT_4, period = 5000, payload = 200, priority = 3, deadline = 10000, jitter = 2500, bandwidth = 0, path = path1
1-th at interval 10
2-th at interval 18
